In [1]:
import datetime as dt

import pandas as pd
import numpy as np
from lenskit.algorithms import item_knn, user_knn
from sklearn.model_selection import ParameterGrid

from recsys4daos.datasets import to_lenskit
from recsys4daos.model_selection import cvtt_open, explore_hparams
from recsys4daos.evaluation import test_with_hparams_lenskit

import paths

/home/daviddavo/recsys4daos/src/recsys4daos/model_selection.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Parameters

In [2]:
# Dataset config
ORG_NAME = 'Decentraland'
SPLITS_FREQ = 'W-THU'  # Split weekly
LAST_FOLDS = 20  # Use just last 10 splits
SPLITS_NORMALIZE = True

# Evaluation
K_RECOMMENDATIONS: list[int] = [1,3,5,10,15,100]
SPLITS_FREQ: str = 'W-THU' # split weekly
SPLITS_NORMALIZE = True # Wether or not to move everything to 00:00
LAST_FOLDS = 10 # Use just last 10 splits
LAST_FOLD_DATE_STR: str = None

# Search space config
WINDOW_SIZES = ['7d', '14d', '21d', '30d', '60d', '90d', '10YE']
ITEMKNN_Ks = [1,2,3,4,5,6,7,8,9,10,15]

OPTIM_METRIC = 'map@10'

In [3]:
# Parameters
EXECUTION_ID = "2024-09-04T10:00"
WINDOW_SIZES = ["1d", "7d", "14d"]
MODEL_NAMES = ["all-MiniLM-L12-v2"]
ORG_NAME = "Aave - Aavegotchi"
SPLITS_FREQ = "5d"
LAST_FOLDS = 10
SPLITS_NORMALIZE = True
LAST_FOLD_DATE_STR = "2023-05-01"


# Load the dataset

In [4]:
dfp = paths.load_proposals(ORG_NAME)
dfv = paths.load_votes(ORG_NAME)

print(dfp.info())
print(dfv.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1081 entries, 0 to 1080
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 1081 non-null   object        
 1   author             1081 non-null   object        
 2   date               1081 non-null   datetime64[us]
 3   start              1081 non-null   datetime64[us]
 4   end                1081 non-null   datetime64[us]
 5   platform_proposal  1081 non-null   object        
dtypes: datetime64[us](3), object(3)
memory usage: 50.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2328356 entries, 0 to 2328355
Data columns (total 4 columns):
 #   Column    Dtype         
---  ------    -----         
 0   id        object        
 1   proposal  object        
 2   voter     object        
 3   date      datetime64[us]
dtypes: datetime64[us](1), object(3)
memory usage: 71.1+ MB
None


In [5]:
df = to_lenskit(dfv)
df

,user,item,timestamp,rating
0,0x7e741fbdc71b8fa2310e9836146296c327b79dce-vot...,05fecd7e-ef0d-5804-a509-7c9bde951705,2019-12-13 15:27:14,1
1,0x7e741fbdc71b8fa2310e9836146296c327b79dce-vot...,05fecd7e-ef0d-5804-a509-7c9bde951705,2019-12-13 15:29:30,1
2,0x7e741fbdc71b8fa2310e9836146296c327b79dce-vot...,05fecd7e-ef0d-5804-a509-7c9bde951705,2019-12-13 15:31:45,1
3,0x7e741fbdc71b8fa2310e9836146296c327b79dce-vot...,a0847e79-6ad8-58ec-b525-d8dbff21b7af,2020-01-13 13:14:17,1
4,0x7e741fbdc71b8fa2310e9836146296c327b79dce-vot...,a0847e79-6ad8-58ec-b525-d8dbff21b7af,2020-01-13 13:17:14,1
...,...,...,...,...
2328351,0x4da0878ddd74208c1e67915f5c25db99b22ffdd1,a6bc61cd-cc0d-57f1-8c26-ecd7027eb461,2023-07-20 16:28:43,1
2328352,0x2415b3876ceb4bfd0c90467419039979b3ff7daa,a6bc61cd-cc0d-57f1-8c26-ecd7027eb461,2023-07-20 16:30:21,1
2328353,0x2975e77ff2954f27e0b18403ccf02c528053ae4c,a6bc61cd-cc0d-57f1-8c26-ecd7027eb461,2023-07-20 16:32:03,1
2328354,0xb7b827186385ed69023ed824fc8950ef07731be9,a6bc61cd-cc0d-57f1-8c26-ecd7027eb461,2023-07-20 16:33:32,1


## Split in folds

In [6]:
all_folds = { f.end:f for f in cvtt_open(
    df, SPLITS_FREQ, dfp.reset_index(), remove_not_in_train_col='item', col_item='item', last_fold=LAST_FOLD_DATE_STR,
)}
last_folds_idx = list(all_folds.keys())[-LAST_FOLDS:]
last_folds_idx

[Timestamp('2023-03-17 00:00:00'),
 Timestamp('2023-03-22 00:00:00'),
 Timestamp('2023-03-27 00:00:00'),
 Timestamp('2023-04-01 00:00:00'),
 Timestamp('2023-04-06 00:00:00'),
 Timestamp('2023-04-11 00:00:00'),
 Timestamp('2023-04-16 00:00:00'),
 Timestamp('2023-04-21 00:00:00'),
 Timestamp('2023-04-26 00:00:00'),
 Timestamp('2023-05-01 00:00:00')]

# Item-based KNN

In [7]:
def testHParamsItemKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = item_knn.ItemItem(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsItemKNN(last_folds_idx[-1],5,'14d'))

/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
Numba is using threading layer omp - consider TBB


found 1 potential runtime problems - see https://boi.st/lkpy-perf


/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/lenskit/algorithms/item_knn.py:119: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  b = blocks[bi]


fold_t             2023-05-01 00:00:00
time_train                      5.6701
time_rec                      2.319076
open_proposals                       6
min_recs                             1
avg_recs                      3.916538
precision@1                   0.541473
precision@3                   0.440323
precision@5                   0.322411
precision@10                  0.161669
precision@15                  0.107779
precision@100                 0.016167
ndcg@1                         0.33958
ndcg@3                        0.459626
ndcg@5                        0.503971
ndcg@10                       0.504467
ndcg@15                       0.504467
ndcg@100                      0.504467
map@1                         0.541473
map@3                         0.687232
map@5                         0.734389
map@10                        0.734904
map@15                        0.734904
map@100                       0.734904
recall@1                      0.383582
recall@3                 

## Exploring hparams

In [8]:
results = explore_hparams(
    testHParamsItemKNN, 
    ParameterGrid({
    'fold': last_folds_idx,
    'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
    'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('itemknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfi = pd.DataFrame(results)
mdfi

Restored checkpoint from ../.cache/Aave - Aavegotchi/hparams-itemknn_5d_normalize.pkl with 880 results


  0%|          | 0/330 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2023-03-17,1,7d,2023-03-17,0.115571,3.020357,5,2,4.680390,0.770675,...,1.0,1.0,1.0,0.579506,0.943832,0.954392,0.954392,0.954392,0.954392,0.783904
1,2023-03-17,1,14d,2023-03-17,0.196827,3.848087,5,2,4.736057,0.835371,...,1.0,1.0,1.0,0.606004,0.977470,0.986899,0.986899,0.986899,0.986899,0.878617
2,2023-03-17,2,7d,2023-03-17,0.108950,2.976980,5,2,4.680390,0.770675,...,1.0,1.0,1.0,0.579506,0.943832,0.954392,0.954392,0.954392,0.954392,0.756456
3,2023-03-17,2,14d,2023-03-17,0.180264,3.787646,5,2,4.736057,0.832687,...,1.0,1.0,1.0,0.604913,0.977407,0.986862,0.986862,0.986862,0.986862,0.791465
4,2023-03-17,3,7d,2023-03-17,0.125695,3.255594,5,2,4.680390,0.770675,...,1.0,1.0,1.0,0.579506,0.943832,0.954392,0.954392,0.954392,0.954392,0.755309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,2023-05-01,7,1d,2023-05-01,0.031040,0.461651,6,2,4.624490,0.910204,...,1.0,1.0,1.0,0.849776,0.971852,0.980741,0.980741,0.980741,0.980741,0.279038
326,2023-05-01,8,1d,2023-05-01,0.031342,0.462809,6,2,4.624490,0.910204,...,1.0,1.0,1.0,0.849776,0.971852,0.980741,0.980741,0.980741,0.980741,0.278542
327,2023-05-01,9,1d,2023-05-01,0.031372,0.462298,6,2,4.624490,0.910204,...,1.0,1.0,1.0,0.849776,0.971852,0.980741,0.980741,0.980741,0.980741,0.278883
328,2023-05-01,10,1d,2023-05-01,0.031104,0.460429,6,2,4.624490,0.910204,...,1.0,1.0,1.0,0.849776,0.971852,0.980741,0.980741,0.980741,0.980741,0.278431


### Best overall hparams

In [9]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfi.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfi[mdfi['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train  avg_recs  precision@5  precision@10  \
window_size k                                                     
1d          1     0.010485  6.087857     0.300155      0.162536   
            3     0.010860  6.087857     0.300179      0.162525   
            2     0.009778  6.087857     0.300176      0.162525   
            6     0.010151  6.087857     0.300148      0.162525   
            7     0.009947  6.087857     0.300116      0.162525   
            5     0.010121  6.087857     0.300179      0.162525   
            8     0.009755  6.087857     0.300116      0.162525   
            4     0.010823  6.087857     0.300179      0.162525   
            9     0.009756  6.087857     0.300116      0.162525   
            10    0.010519  6.087857     0.300116      0.162525   
            15    0.009668  6.087857     0.300116      0.162525   
7d          1     0.080905  6.269580     0.360045      0.187356   
            2     0.080178  6.269580     0.359984      0.187358   
            3     0.078571  6.269580     0.359920      0.187358   
            5     0.077457  6.269580     0.359762      0.187358   
            6     0.075993  6.269580     0.359759      0.187358   
            7     0.072302  6.269580     0.359747      0.187358   
            4     0.076788  6.269580     0.359836      0.187359   
            8     0.072783  6.269580     0.359750      0.187358   
            9     0.076242  6.269580     0.359747      0.187358   
            10    0.078886  6.269580     0.359744      0.187358   
            15    0.073600  6.269580     0.359741      0.187358   
14d         2     0.161543  6.526166     0.366687      0.192749   
            3     0.155454  6.526166     0.366714      0.192746   
            4     0.153324  6.526166     0.366683      0.192744   
            5     0.159181  6.526166     0.366652      0.192744   
            6     0.161861  6.526166     0.366668      0.192744   
            15    0.154732  6.526166     0.366681      0.192745   
            8     0.153766  6.526166     0.366683      0.192745   
            10    0.155833  6.526166     0.366684      0.192745   
            9     0.153412  6.526166     0.366664      0.192745   
            7     0.160215  6.526166     0.366674      0.192744   
            1     0.156464  6.526166     0.366746      0.192754   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
1d          1        0.016322  0.053603  0.054430  0.054503  0.819849   
            3        0.016322  0.053502  0.054323  0.054396  0.817948   
            2        0.016322  0.053507  0.054333  0.054407  0.817619   
            6        0.016322  0.053497  0.054319  0.054393  0.817323   
            7        0.016322  0.053493  0.054318  0.054391  0.817257   
            5        0.016322  0.053498  0.054318  0.054391  0.817297   
            8        0.016322  0.053492  0.054316  0.054390  0.817218   
            4        0.016322  0.053497  0.054317  0.054390  0.817216   
            9        0.016322  0.053485  0.054309  0.054383  0.816998   
            10       0.016322  0.053485  0.054309  0.054383  0.816998   
            15       0.016322  0.053484  0.054309  0.054382  0.816972   
7d          1        0.018769  0.572376  0.576954  0.577106  0.791257   
            2        0.018769  0.570193  0.574803  0.574952  0.785501   
            3        0.018769  0.570059  0.574731  0.574880  0.785248   
            5        0.018769  0.569887  0.574656  0.574805  0.784906   
            6        0.018769  0.569845  0.574617  0.574766  0.784835   
            7        0.018769  0.569823  0.574600  0.574750  0.784795   
            4        0.018769  0.569882  0.574607  0.574755  0.784819   
            8        0.018769  0.569798  0.574572  0.574721  0.784755   
            9        0.018769  0.569747  0.574523  0.574672  0.784696   
            10       0.018769  0.569738  0.574519  0.574668  0.7

Now let's see the behaviour in each fold

In [10]:
best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'itemknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Aave - Aavegotchi/models/itemknn-best-avg_5d_normalize.parquet


/tmp/ipykernel_3824813/3951593685.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2023-03-17,1d,1,2023-03-17,0.016810,0.446253,5,2,3.434018,0.800587,0.512219,0.309091,0.154545,...,1.000000,1.0,1.0,0.722527,0.948675,0.949503,0.949503,0.949503,0.949503,0.419196
2023-03-22,1d,1,2023-03-22,0.002659,0.086634,5,1,3.308642,0.543210,0.382716,0.237037,0.118519,...,1.000000,1.0,1.0,0.888258,0.955674,0.960993,0.960993,0.960993,0.960993,0.332744
2023-03-27,1d,1,2023-03-27,0.023725,0.545236,5,1,2.888172,0.806452,0.420789,0.279140,0.139570,...,1.000000,1.0,1.0,0.870044,0.957702,0.986292,0.986292,0.986292,0.986292,0.474173
2023-04-01,1d,1,2023-04-01,0.006566,0.716728,11,1,9.171788,0.567039,0.424115,0.310894,0.187291,...,0.998429,1.0,1.0,0.890421,0.814829,0.834644,0.858021,0.858021,0.858021,2.193944
2023-04-06,1d,1,2023-04-06,0.011496,1.889420,14,2,12.274854,0.735380,0.413418,0.296296,0.169981,...,0.990903,1.0,1.0,0.820132,0.917643,0.920542,0.924925,0.924925,0.924925,2.686121
2023-04-11,1d,1,2023-04-11,0.002686,0.081579,9,1,7.932432,0.756757,0.279279,0.183784,0.132432,...,1.000000,1.0,1.0,0.941071,0.943678,0.950575,0.950575,0.950575,0.950575,0.304934
2023-04-16,1d,1,2023-04-16,0.007774,0.113484,6,2,4.480392,0.352941,0.398693,0.247059,0.123529,...,1.000000,1.0,1.0,0.726852,0.963964,0.963964,0.963964,0.963964,0.963964,0.320958
2023-04-21,1d,1,2023-04-21,0.002972,0.084604,9,2,7.208955,0.731343,0.636816,0.486567,0.261194,...,1.000000,1.0,1.0,0.437293,0.809454,0.919351,0.947712,0.947712,0.947712,0.944623
2023-04-26,1d,1,2023-04-26,0.003385,0.853802,4,1,2.900990,0.944554,0.643894,0.386337,0.193168,...,1.000000,1.0,1.0,0.520790,0.996354,0.996354,0.996354,0.996354,0.996354,2.963654


### Best hparams by fold

This are the parameters used to check which model is the best (validation)

In [11]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfi.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'itemknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/Aave - Aavegotchi/models/itemknn-best-val_5d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2023-03-17,1,14d,0.196827,4.736057,0.350313,0.175157,0.017516,0.800985,0.800985,0.800985,0.911646,0.911646,0.911646,1.000000,1.000000,1.0,0.986899,0.986899,0.986899
2023-03-22,1,1d,0.002659,3.308642,0.237037,0.118519,0.011852,0.020462,0.020462,0.020462,0.758916,0.758916,0.758916,1.000000,1.000000,1.0,0.960993,0.960993,0.960993
2023-03-27,1,1d,0.023725,2.888172,0.279140,0.139570,0.013957,0.112092,0.112092,0.112092,0.898781,0.898781,0.898781,1.000000,1.000000,1.0,0.986292,0.986292,0.986292
2023-04-01,1,1d,0.006566,9.171788,0.310894,0.187291,0.018771,0.034124,0.035204,0.035229,0.753566,0.778648,0.779029,0.933788,0.998429,1.0,0.834644,0.858021,0.858021
2023-04-06,1,1d,0.011496,12.274854,0.296296,0.169981,0.017573,0.117911,0.120787,0.121412,0.813908,0.831238,0.835536,0.945032,0.990903,1.0,0.920542,0.924925,0.924925
2023-04-11,1,7d,0.110449,6.062630,0.307307,0.193737,0.019374,0.437242,0.463709,0.463709,0.804863,0.835830,0.835830,0.864857,1.000000,1.0,0.932526,0.951404,0.951404
2023-04-16,1,7d,0.073312,4.960000,0.265200,0.132700,0.013270,0.318259,0.318259,0.318259,0.741747,0.741747,0.741747,0.999833,1.000000,1.0,0.969113,0.969419,0.969419
2023-04-21,4,7d,0.044719,8.929205,0.209828,0.107847,0.010785,0.909230,0.910612,0.910612,0.993029,0.994384,0.994384,0.994447,1.000000,1.0,0.993686,0.996189,0.996189
2023-04-26,1,7d,0.037386,3.904501,0.564318,0.282159,0.028216,0.829222,0.829222,0.829222,0.980011,0.980011,0.980011,1.000000,1.000000,1.0,0.987225,0.987225,0.987225


In [12]:
best_hparams.tail(len(best_hparams)-1).describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.0,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2023-04-11 00:00:00,0.037978,3.554925,7.666667,1.222222,6.236031,0.756020,0.468492,0.304923,0.163216,...,0.998815,1.0,1.0,0.793601,0.941447,0.951751,0.957246,0.957246,0.957246,1.782973
min,2023-03-22 00:00:00,0.002659,0.086634,4.000000,1.000000,2.888172,0.537000,0.340898,0.209828,0.107847,...,0.990903,1.0,1.0,0.363299,0.814829,0.834644,0.858021,0.858021,0.858021,0.332744
25%,2023-04-01 00:00:00,0.011496,0.545236,5.000000,1.000000,3.904501,0.567039,0.413418,0.265200,0.132700,...,1.000000,1.0,1.0,0.737063,0.917643,0.932526,0.951404,0.951404,0.951404,0.375884
50%,2023-04-11 00:00:00,0.031492,1.186623,6.000000,1.000000,4.960000,0.754697,0.421769,0.279140,0.139570,...,1.000000,1.0,1.0,0.849776,0.957702,0.969113,0.969419,0.969419,0.969419,0.474173
75%,2023-04-21 00:00:00,0.044719,1.889420,9.000000,1.000000,8.929205,0.910204,0.430667,0.307307,0.187291,...,1.000000,1.0,1.0,0.888258,0.971852,0.986292,0.986292,0.986292,0.986292,2.686121
max,2023-05-01 00:00:00,0.110449,13.993475,14.000000,2.000000,12.274854,0.991211,0.937033,0.564318,0.282159,...,1.000000,1.0,1.0,0.989245,0.990452,0.993686,0.996189,0.996189,0.996189,5.347127
std,NaN,0.034870,5.300751,3.316625,0.440959,3.197820,0.177562,0.178400,0.102612,0.053547,...,0.003012,0.0,0.0,0.179653,0.054726,0.050615,0.043153,0.043153,0.043153,1.856455


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb).

In [13]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfi.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'itemknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams[display_columns]

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Aave - Aavegotchi/models/itemknn-best-test_5d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2023-03-22,1,14d,0.204176,4.412242,0.270126,0.135063,0.013506,0.539107,0.539107,0.539107,0.610740,0.610740,0.610740,1.000000,1.000000,1.0,0.910031,0.910031,0.910031
2023-03-27,1,1d,0.023725,2.888172,0.279140,0.139570,0.013957,0.112092,0.112092,0.112092,0.898781,0.898781,0.898781,1.000000,1.000000,1.0,0.986292,0.986292,0.986292
2023-04-01,1,1d,0.006566,9.171788,0.310894,0.187291,0.018771,0.034124,0.035204,0.035229,0.753566,0.778648,0.779029,0.933788,0.998429,1.0,0.834644,0.858021,0.858021
2023-04-06,1,1d,0.011496,12.274854,0.296296,0.169981,0.017573,0.117911,0.120787,0.121412,0.813908,0.831238,0.835536,0.945032,0.990903,1.0,0.920542,0.924925,0.924925
2023-04-11,1,1d,0.002686,7.932432,0.183784,0.132432,0.013243,0.031293,0.034768,0.034768,0.751599,0.798548,0.798548,0.783333,1.000000,1.0,0.950575,0.950575,0.950575
2023-04-16,1,7d,0.073312,4.960000,0.265200,0.132700,0.013270,0.318259,0.318259,0.318259,0.741747,0.741747,0.741747,0.999833,1.000000,1.0,0.969113,0.969419,0.969419
2023-04-21,1,7d,0.043766,8.929205,0.208933,0.107847,0.010785,0.908325,0.910414,0.910414,0.992351,0.994128,0.994128,0.992651,1.000000,1.0,0.994363,0.996829,0.996829
2023-04-26,4,7d,0.034235,3.904501,0.564318,0.282159,0.028216,0.828534,0.828534,0.828534,0.978936,0.978936,0.978936,1.000000,1.000000,1.0,0.986920,0.986920,0.986920
2023-05-01,1,7d,0.083066,3.650203,0.314817,0.157525,0.015752,0.460570,0.460641,0.460641,0.767714,0.767811,0.767811,0.999613,1.000000,1.0,0.897791,0.897925,0.897925


In [14]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.0,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2023-04-11 00:00:00,0.053670,3.809022,7.666667,1.111111,6.458155,0.698981,0.458814,0.299279,0.160507,...,0.998815,1.0,1.0,0.779197,0.930909,0.938919,0.942326,0.942326,0.942326,1.793466
min,2023-03-22 00:00:00,0.002686,0.081579,4.000000,1.000000,2.888172,0.346980,0.279279,0.183784,0.107847,...,0.990903,1.0,1.0,0.362936,0.814829,0.834644,0.858021,0.858021,0.858021,0.304934
25%,2023-04-01 00:00:00,0.011496,0.716728,5.000000,1.000000,3.904501,0.567039,0.413418,0.265200,0.132700,...,1.000000,1.0,1.0,0.730566,0.907653,0.910031,0.910031,0.910031,0.910031,0.474173
50%,2023-04-11 00:00:00,0.034235,1.811741,6.000000,1.000000,4.960000,0.735380,0.424115,0.279140,0.139570,...,1.000000,1.0,1.0,0.820132,0.943678,0.950575,0.950575,0.950575,0.950575,0.642445
75%,2023-04-21 00:00:00,0.073312,2.808040,9.000000,1.000000,8.929205,0.806452,0.434671,0.310894,0.169981,...,1.000000,1.0,1.0,0.890421,0.968043,0.986292,0.986292,0.986292,0.986292,2.686121
max,2023-05-01 00:00:00,0.204176,13.960208,14.000000,2.000000,12.274854,0.991530,0.937056,0.564318,0.282159,...,1.000000,1.0,1.0,0.989094,0.993136,0.994363,0.996829,0.996829,0.996829,5.230372
std,NaN,0.063118,5.105729,3.316625,0.333333,3.222152,0.208590,0.187484,0.108714,0.051278,...,0.003012,0.0,0.0,0.187490,0.055830,0.052923,0.047590,0.047590,0.047590,1.769896


# User-based KNN

In [15]:
def testHParamsUserKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = user_knn.UserUser(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsUserKNN(last_folds_idx[-1],5,'14d'))

fold_t             2023-05-01 00:00:00
time_train                    0.713642
time_rec                       3.54537
open_proposals                       6
min_recs                             1
avg_recs                      3.916538
precision@1                   0.550747
precision@3                   0.432251
precision@5                   0.305719
precision@10                  0.161669
precision@15                  0.107779
precision@100                 0.016167
ndcg@1                        0.345396
ndcg@3                        0.447543
ndcg@5                        0.482727
ndcg@10                       0.496396
ndcg@15                       0.496396
ndcg@100                      0.496396
map@1                         0.550747
map@3                          0.65984
map@5                         0.699956
map@10                        0.714381
map@15                        0.714381
map@100                       0.714381
recall@1                      0.389808
recall@3                 

## Exploring hparams

In [16]:
results = explore_hparams(
    testHParamsUserKNN, 
    ParameterGrid({
        'fold': last_folds_idx,
        'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
        'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('userknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfu = pd.DataFrame(results)
mdfu

Restored checkpoint from ../.cache/Aave - Aavegotchi/hparams-userknn_5d_normalize.pkl with 880 results


  0%|          | 0/330 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2023-03-17,1,7d,2023-03-17,0.092943,3.154488,5,2,4.680390,0.780426,...,1.0,1.0,1.0,0.584837,0.979223,0.989649,0.989649,0.989649,0.989649,0.811882
1,2023-03-17,1,14d,2023-03-17,0.166426,4.554055,5,2,4.736057,0.779600,...,1.0,1.0,1.0,0.582536,0.956978,0.966978,0.966978,0.966978,0.966978,0.808106
2,2023-03-17,2,7d,2023-03-17,0.091827,3.248714,5,2,4.680390,0.780787,...,1.0,1.0,1.0,0.585029,0.978327,0.988748,0.988748,0.988748,0.988748,0.729675
3,2023-03-17,2,14d,2023-03-17,0.166861,4.561153,5,2,4.736057,0.781092,...,1.0,1.0,1.0,0.582888,0.958676,0.968672,0.968672,0.968672,0.968672,0.809753
4,2023-03-17,3,7d,2023-03-17,0.091649,3.151829,5,2,4.680390,0.784399,...,1.0,1.0,1.0,0.586940,0.970756,0.981131,0.981131,0.981131,0.981131,0.731467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,2023-05-01,7,1d,2023-05-01,0.031048,0.371717,6,2,4.624490,0.934694,...,1.0,1.0,1.0,0.847889,0.950938,0.950938,0.950938,0.950938,0.950938,0.368366
326,2023-05-01,8,1d,2023-05-01,0.031296,0.378951,6,2,4.624490,0.934694,...,1.0,1.0,1.0,0.847889,0.946609,0.955267,0.955267,0.955267,0.955267,0.370478
327,2023-05-01,9,1d,2023-05-01,0.031474,0.376810,6,2,4.624490,0.934694,...,1.0,1.0,1.0,0.847889,0.946609,0.946609,0.946609,0.946609,0.946609,0.280320
328,2023-05-01,10,1d,2023-05-01,0.031113,0.372184,6,2,4.624490,0.934694,...,1.0,1.0,1.0,0.847889,0.946609,0.946609,0.946609,0.946609,0.946609,0.276152


### Best overall hparams

In [17]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfu.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfu[mdfu['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train  avg_recs  precision@5  precision@10  \
window_size k                                                     
1d          7     0.009214  6.087857     0.298389      0.162125   
            8     0.009271  6.087857     0.298379      0.162125   
            6     0.009213  6.087857     0.298566      0.162125   
            5     0.009265  6.087857     0.298149      0.162146   
            9     0.009307  6.087857     0.298258      0.162136   
            10    0.009194  6.087857     0.298052      0.162136   
            4     0.009363  6.087857     0.298066      0.162140   
            3     0.009231  6.087857     0.298257      0.162151   
14d         1     0.134166  6.526166     0.362316      0.192608   
1d          2     0.009260  6.087857     0.299364      0.162157   
            15    0.009164  6.087857     0.298007      0.162184   
14d         4     0.133707  6.526166     0.364807      0.192683   
            3     0.133425  6.526166     0.364303      0.192676   
            2     0.133928  6.526166     0.363842      0.192634   
            5     0.133632  6.526166     0.364548      0.192678   
            6     0.133900  6.526166     0.364567      0.192684   
            7     0.133514  6.526166     0.365406      0.192684   
            8     0.133753  6.526166     0.365429      0.192683   
            9     0.133619  6.526166     0.365412      0.192683   
            10    0.133923  6.526166     0.365364      0.192684   
            15    0.134133  6.526166     0.365817      0.192674   
7d          10    0.062673  6.269580     0.355862      0.187122   
            9     0.062978  6.269580     0.355900      0.187118   
            15    0.062540  6.269580     0.356425      0.187084   
1d          1     0.009247  6.087857     0.298514      0.162151   
7d          3     0.062439  6.269580     0.354500      0.187137   
            4     0.062509  6.269580     0.356350      0.187127   
            6     0.062819  6.269580     0.356620      0.187114   
            5     0.062547  6.269580     0.356627      0.187124   
            7     0.062910  6.269580     0.356379      0.187108   
            8     0.062959  6.269580     0.356471      0.187114   
            2     0.062817  6.269580     0.353543      0.187144   
            1     0.062808  6.269580     0.353529      0.187143   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
1d          7        0.016322  0.055019  0.055607  0.055734  0.840972   
            8        0.016322  0.055008  0.055599  0.055725  0.840729   
            6        0.016322  0.054904  0.055480  0.055607  0.839929   
            5        0.016322  0.054897  0.055481  0.055606  0.839091   
            9        0.016322  0.054915  0.055518  0.055643  0.837870   
            10       0.016322  0.054877  0.055488  0.055613  0.837070   
            4        0.016322  0.053885  0.054464  0.054590  0.829842   
            3        0.016322  0.053788  0.054364  0.054488  0.826837   
14d         1        0.019313  0.710430  0.720701  0.721041  0.821628   
1d          2        0.016322  0.053307  0.053885  0.054008  0.817702   
            15       0.016322  0.053321  0.053924  0.054044  0.800485   
14d         4        0.019313  0.694602  0.703201  0.703468  0.801534   
            3        0.019313  0.694207  0.703143  0.703419  0.800851   
            2        0.019313  0.694123  0.703242  0.703556  0.799859   
            5        0.019313  0.693920  0.702673  0.702945  0.800596   
            6        0.019313  0.693503  0.702274  0.702542  0.800027   
            7        0.019313  0.692852  0.701093  0.701362  0.798924   
            8        0.019313  0.692320  0.700600  0.700870  0.798243   
            9        0.019313  0.691093  0.699384  0.699652  0.796042   
            10       0.019313  0.690715  0.698997  0.699264  0.795608   
            15       0.019313  0.689674  0.697629  0.697905  0.7

In [18]:
best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'userknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Aave - Aavegotchi/models/userknn-best-avg_5d_normalize.parquet


/tmp/ipykernel_3824813/3337470013.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2023-03-17,1d,7,2023-03-17,0.016248,0.500813,5,2,3.434018,0.800587,0.512219,0.309091,0.154545,...,1.000000,1.0,1.0,0.722527,0.948675,0.949503,0.949503,0.949503,0.949503,0.424116
2023-03-22,1d,7,2023-03-22,0.001814,0.072112,5,1,3.308642,0.703704,0.386831,0.237037,0.118519,...,1.000000,1.0,1.0,0.907895,0.964689,0.968927,0.968927,0.968927,0.968927,0.248228
2023-03-27,1d,7,2023-03-27,0.024042,0.531804,5,1,2.888172,0.935484,0.420072,0.279140,0.139570,...,1.000000,1.0,1.0,0.881073,0.948174,0.973744,0.973744,0.973744,0.973744,0.479283
2023-04-01,1d,7,2023-04-01,0.004324,0.534633,11,1,9.171788,0.678771,0.439013,0.314246,0.187291,...,0.998429,1.0,1.0,0.830853,0.816135,0.836225,0.856777,0.856777,0.856777,2.260388
2023-04-06,1d,7,2023-04-06,0.011295,1.645272,14,2,12.274854,0.729045,0.405945,0.285965,0.166277,...,0.982760,1.0,1.0,0.825532,0.877642,0.868037,0.873132,0.873132,0.873132,2.755397
2023-04-11,1d,7,2023-04-11,0.001939,0.067453,9,1,7.932432,0.851351,0.373874,0.240541,0.132432,...,1.000000,1.0,1.0,0.892063,0.900000,0.905882,0.905882,0.905882,0.905882,0.313459
2023-04-16,1d,7,2023-04-16,0.003432,0.096033,6,2,4.480392,0.411765,0.401961,0.247059,0.123529,...,1.000000,1.0,1.0,0.765873,0.968992,0.968992,0.968992,0.968992,0.968992,0.239133
2023-04-21,1d,7,2023-04-21,0.002301,0.071947,9,2,7.208955,0.791045,0.547264,0.420896,0.261194,...,1.000000,1.0,1.0,0.551460,0.759544,0.802798,0.826905,0.826905,0.826905,0.975061
2023-04-26,1d,7,2023-04-26,0.002733,0.806198,4,1,2.900990,0.970297,0.643894,0.386337,0.193168,...,1.000000,1.0,1.0,0.533503,0.595842,0.595842,0.595842,0.595842,0.595842,2.974269


### Best hparams by fold

In [19]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfu.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'userknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/Aave - Aavegotchi/models/userknn-best-val_5d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2023-03-17,1,1d,0.016904,3.434018,0.309091,0.154545,0.015455,0.082485,0.082485,0.082485,0.928311,0.928311,0.928311,1.000000,1.000000,1.0,0.952194,0.952194,0.952194
2023-03-22,4,1d,0.001845,3.308642,0.237037,0.118519,0.011852,0.022283,0.022283,0.022283,0.856653,0.856653,0.856653,1.000000,1.000000,1.0,0.954301,0.954301,0.954301
2023-03-27,9,1d,0.023695,2.888172,0.279140,0.139570,0.013957,0.117587,0.117587,0.117587,0.959450,0.959450,0.959450,1.000000,1.000000,1.0,0.987414,0.987414,0.987414
2023-04-01,15,1d,0.004319,9.171788,0.313408,0.187291,0.018771,0.036214,0.037211,0.037235,0.812560,0.836451,0.836832,0.938024,0.998429,1.0,0.839092,0.860110,0.860110
2023-04-06,5,1d,0.011222,12.274854,0.285478,0.166326,0.017573,0.118878,0.121939,0.123051,0.824950,0.841839,0.848807,0.934311,0.982830,1.0,0.866625,0.871889,0.871889
2023-04-11,10,1d,0.001961,7.932432,0.237838,0.132432,0.013243,0.036727,0.037859,0.037859,0.874444,0.893299,0.893299,0.931532,1.000000,1.0,0.905882,0.905882,0.905882
2023-04-16,5,7d,0.056716,4.960000,0.264800,0.132700,0.013270,0.361955,0.362107,0.362107,0.883369,0.883703,0.883703,0.999000,1.000000,1.0,0.896927,0.897122,0.897122
2023-04-21,3,14d,0.073985,8.932813,0.213468,0.111997,0.011200,0.956878,0.959030,0.959030,0.991049,0.993271,0.993271,0.991773,1.000000,1.0,0.990312,0.994316,0.994316
2023-04-26,1,14d,0.090106,3.914707,0.556284,0.278142,0.027814,0.924396,0.924396,0.924396,0.972625,0.972625,0.972625,1.000000,1.000000,1.0,0.975708,0.975708,0.975708


In [20]:
best_hparams.tail(len(best_hparams)-1).describe()[display_columns]

,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.0,9.000000,9.000000,9.000000
mean,0.032744,6.445322,0.295749,0.156013,0.015710,0.294661,0.295494,0.295620,0.903507,0.910417,0.911234,0.977182,0.997918,1.0,0.929309,0.932696,0.932696
min,0.001845,2.888172,0.213468,0.111997,0.011200,0.022283,0.022283,0.022283,0.812560,0.836451,0.836832,0.931532,0.982830,1.0,0.839092,0.860110,0.860110
25%,0.004319,3.914707,0.237838,0.132432,0.013243,0.036727,0.037859,0.037859,0.856653,0.856653,0.856653,0.938024,1.000000,1.0,0.896927,0.897122,0.897122
50%,0.023695,4.960000,0.274286,0.137143,0.013714,0.117587,0.117587,0.117587,0.883369,0.893299,0.893299,0.999000,1.000000,1.0,0.947518,0.947518,0.947518
75%,0.056716,8.932813,0.285478,0.166326,0.017573,0.361955,0.362107,0.362107,0.959450,0.959450,0.959450,1.000000,1.000000,1.0,0.975708,0.975708,0.975708
max,0.090106,12.274854,0.556284,0.278142,0.027814,0.956878,0.959030,0.959030,0.991049,0.993271,0.993271,1.000000,1.000000,1.0,0.990312,0.994316,0.994316
std,0.033213,3.244878,0.102201,0.051319,0.005168,0.380256,0.380372,0.380307,0.067327,0.060550,0.059541,0.032068,0.005682,0.0,0.054626,0.050347,0.050347


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb)

In [21]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfu.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'userknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Aave - Aavegotchi/models/userknn-best-test_5d_normalize.parquet


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,k,window_size,,,,,,,,,,,,,,,,,,,,,
2023-03-22,1,1d,2023-03-22,0.001865,0.071020,5,1,3.308642,0.716049,0.374486,0.237037,0.118519,...,1.000000,1.0,1.0,0.918103,0.957650,0.961749,0.961749,0.961749,0.961749,0.249828
2023-03-27,4,1d,2023-03-27,0.023629,0.524175,5,1,2.888172,0.896774,0.415054,0.279140,0.139570,...,1.000000,1.0,1.0,0.883133,0.946838,0.973068,0.973068,0.973068,0.973068,0.663580
2023-04-01,9,1d,2023-04-01,0.004323,0.535735,11,1,9.171788,0.675978,0.439013,0.314525,0.187291,...,0.998429,1.0,1.0,0.831807,0.816635,0.836754,0.857862,0.857862,0.857862,2.269770
2023-04-06,15,1d,2023-04-06,0.011268,1.754557,14,2,12.274854,0.726608,0.405783,0.286062,0.166813,...,0.983676,1.0,1.0,0.820990,0.919941,0.914514,0.918819,0.918819,0.918819,2.674053
2023-04-11,5,1d,2023-04-11,0.001941,0.067432,9,1,7.932432,0.851351,0.378378,0.243243,0.132432,...,1.000000,1.0,1.0,0.892063,0.902941,0.905882,0.905882,0.905882,0.905882,0.221546
2023-04-16,10,1d,2023-04-16,0.003422,0.097403,6,2,4.480392,0.401961,0.401961,0.247059,0.123529,...,1.000000,1.0,1.0,0.760163,0.968254,0.968254,0.968254,0.968254,0.968254,0.239256
2023-04-21,5,7d,2023-04-21,0.030141,11.878622,9,1,8.929205,0.137515,0.338287,0.209125,0.107847,...,1.000000,1.0,1.0,0.933827,0.938009,0.947713,0.964562,0.964562,0.964562,3.658426
2023-04-26,3,14d,2023-04-26,0.090373,17.895483,4,1,3.914707,0.934111,0.921780,0.556284,0.278142,...,1.000000,1.0,1.0,0.366757,0.976116,0.978019,0.978019,0.978019,0.978019,5.192371
2023-05-01,1,14d,2023-05-01,0.108579,2.618931,6,1,3.916538,0.727460,0.421089,0.305100,0.161669,...,1.000000,1.0,1.0,0.711898,0.842003,0.854693,0.855023,0.855023,0.855023,0.462896


In [22]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.0,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2023-04-11 00:00:00,0.030616,3.938151,7.666667,1.222222,6.312970,0.674201,0.455092,0.297508,0.157312,...,0.998012,1.0,1.0,0.790971,0.918710,0.926738,0.931471,0.931471,0.931471,1.736859
min,2023-03-22 00:00:00,0.001865,0.067432,4.000000,1.000000,2.888172,0.137515,0.338287,0.209125,0.107847,...,0.983676,1.0,1.0,0.366757,0.816635,0.836754,0.855023,0.855023,0.855023,0.221546
25%,2023-04-01 00:00:00,0.003422,0.097403,5.000000,1.000000,3.914707,0.675978,0.378378,0.243243,0.123529,...,1.000000,1.0,1.0,0.760163,0.902941,0.905882,0.905882,0.905882,0.905882,0.249828
50%,2023-04-11 00:00:00,0.011268,0.535735,6.000000,1.000000,4.480392,0.726608,0.405783,0.279140,0.139570,...,1.000000,1.0,1.0,0.831807,0.938009,0.947713,0.961749,0.961749,0.961749,0.663580
75%,2023-04-21 00:00:00,0.030141,2.618931,9.000000,1.000000,8.929205,0.851351,0.421089,0.305100,0.166813,...,1.000000,1.0,1.0,0.892063,0.957650,0.968254,0.968254,0.968254,0.968254,2.674053
max,2023-05-01 00:00:00,0.108579,17.895483,14.000000,2.000000,12.274854,0.934111,0.921780,0.556284,0.278142,...,1.000000,1.0,1.0,0.933827,0.976116,0.978019,0.978019,0.978019,0.978019,5.192371
std,NaN,0.040547,6.444831,3.316625,0.440959,3.331523,0.254450,0.177516,0.102919,0.052003,...,0.005401,0.0,0.0,0.175004,0.055883,0.052485,0.049167,0.049167,0.049167,1.813852
